In [3]:
import import_ipynb

from category_list_clean import category_clean_data
from channel_detail_csv import channel_details_csv

In [4]:
#Calling code to retrieve the cleaned category list
category_list = category_clean_data()

category_list

,Category_id,Category_Name
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


In [6]:
#Channel details from csv file
channel_info =channel_details_csv()
channel_info

,channelid,videocategoryid,channelviewcount,videocount,subscribercount
0,UCdzU3DSGzyWzN2118yd9X9g,22,14654,30,18
1,UC0UnhAG47DRyVZGVcbhAXhQ,10,105909,51,184
2,UCXjtAvK5P3wXBGh0vbGylzg,27,48265,72,338
3,UCeKHMeUlcLNPLCLUfZUQI2w,26,2116722,172,22051
4,UCNWPDyaWf2eAHnofFLSnEMg,20,1649075,2777,220
...,...,...,...,...,...
995,UC2l2CRfsl73FV5EETuNcNhg,10,23572,81,10
996,UCjaSeKdnPGviZREF6EcnJcw,24,390440,35,2155
997,UCD-9G8Ryrq9o7M3fEWk5rsA,22,2521,1,8
998,UCXbBAdHOCyfUd6ruv_RUOPQ,20,86773,198,126
